In [ ]:
!pip install python-sat[pblib,aiger]
!pip install python-Levenshtein

# value correspondance



In [ ]:
#### or:
# # import the enchant module
# import enchant
  
# # determining the values of the parameters
# string1 = "abc"
# string2 = "aef"
  
# # the Levenshtein distance between
# # string1 and string2
# print(enchant.utils.levenshtein(string1, string2))

     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149813 sha256=3011bd71ee66a15e197cbfa404f31414d74f7077377e2a87a8209e19a5e5f0b9
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
# sum((len(v) for v in newSchema.values()))
class schema:
  def __init__(self, s):
    self.schema = s
    
    self.cols = {t+'.'+x:self.schema[t][x] for  t in self.schema for x in self.schema[t]}
    self.attr2id = {c:i for i,c in enumerate(self.cols)}
    self.id2attr = {self.attr2id[c]:c for c in self.attr2id}
  def name_to_id(self,a):#if not exist...!!!!!!!!
    return  self.attr2id[a]
  def id_to_name(self,a):#if not exist...!!!!!!!!
    return (self.id2attr[a])
  def id_to_type(self,a):#if not exist...!!!!!!!!
    return (self.cols[self.id2attr[a]])
  def size(self):
    return len(self.attr2id)
oldSchema={'Class':{'ClassId':'int','InstId':'int','TaId':'int'},'Instructor':{'InstId':'int','IName':'string','IPic':'picture'},'TA':{'TaId':'int','TName':'string','TPic':'picture'}}
newSchema={'Class':{'ClassId':'int','InstId':'int','TaId':'int'},'Instructor':{'InstId':'int','IName':'string','PicId':'int'},'TA':{'TaId':'int','TName':'string','PicId':'int'},'Picture':{'PicId':'int','Pic':'picture'}}
s2= schema(newSchema)
s1= schema(oldSchema)
s1.name_to_id('TA.TName')
# s2.size()

7

In [ ]:
from pysat.formula import WCNF
from Levenshtein import distance as levenshtein_distance
from pysat.solvers import Solver
from pysat.examples.rc2 import RC2

#!!!! bug: -0 is meaningless
class val_corr_generator:
  def __init__(self,S1,S2):
    alpha = 15
    size1=S1.size()
    size2=S2.size()
    self.size1=size1
    self.S1=S1
    self.S2=S2

    self.cnf = RC2(WCNF())

    ## hard: variables !!!!!! really needed?
    for i in range (size1*size2):
      self.cnf.add_clause([i,-i])

    ## hard: type-matching-conditions
    for i in range(size1):
      for j in range(size2):
        if S1.id_to_type(i) != S2.id_to_type(j):
          self.cnf.add_clause([-self.twoD2oneD(i,j)])
    
    ## hard: match each i to some j !!!!!!change to a_is that are queried in p
    # js=[x for x in range(size2)]
    for i in range(size1):
      all_poss=[self.twoD2oneD(i,j) for j in range(size2)]
      self.cnf.add_clause(all_poss)

    ## soft: prefer 1-1 to 1-many
    for i in range(size1):
      for j in range(size2):
        for k in range(j,size2):
          self.cnf.add_clause([-self.twoD2oneD(i,j),-self.twoD2oneD(i,k)], weight = alpha)

    ## soft: similar names: more score!!!!!! change similarity maybe..
    for i in range(size1):
      for j in range(size2):
        similarity = 20 - levenshtein_distance(S1.id_to_name(i),S2.id_to_name(j))
        self.cnf.add_clause([self.twoD2oneD(i,j)],similarity)

    # last_sol?(use in update)
    # self.solver = Solver()
    # self.solver.append_formula(self.cnf.clauses, no_return=False)#!!!!!no_return?!
  
  def max_sat_solver(self):
    self.last_solution = self.cnf.compute()
    # self.solver.solve()
    # print(self.solver.get_model())
    # solver.delete()
  def max_sat_updater(self):
    for x in self.last_solution:
      self.cnf.add_clause([-x])
  def return_sol(self):
    matching={}
    for i in self.last_solution:
      if i>0:
        a,b=self.oneD2twoD(i)
        a = self.S1.id_to_name(a)
        b = self.S2.id_to_name(b)
        if a in matching:
          matching[a].append(b)
        else:
          matching[a]=[b]
    print(matching)

  def twoD2oneD(self,i,j):
    return j*self.size1+i
  def oneD2twoD(self,x):
    return (x%self.size1,int(x/self.size1)) 

In [ ]:
phi = val_corr_generator(s1,s2)
a=phi.max_sat_solver()
phi.return_sol()

# phi.max_sat_updater()#!!!! inja akhe ino seda bezanin?:/
# b=phi.max_sat_solver()

{'Class.ClassId': ['Class.InstId'], 'Class.InstId': ['Class.InstId'], 'Class.TaId': ['Class.TaId'], 'Instructor.InstId': ['Instructor.InstId'], 'Instructor.IName': ['Instructor.IName'], 'TA.TaId': ['TA.TaId'], 'TA.TName': ['TA.TName'], 'Instructor.IPic': ['Picture.Pic'], 'TA.TPic': ['Picture.Pic']}
